# Expirement Design for Data Science
## Group 26

### Imports

In [132]:

import pandas as pd
import numpy as np
import os
import initialize
from rank_bm25 import BM25Okapi

### Load Preprocessed Data

In [3]:
trec = initialize.get_TREC45_dataset()

### Step 1 - BM25 Search

In [4]:
trec

,Number,Title,Description,Narrative,Title_tokenized,Description_tokenized,Narrative_tokenized
0,301,International Organized Crime,Identify organizations that participate in int...,A relevant document must as a minimum identify...,"[International, Organized, Crime]","[Identify, organizations, participate, interna...","[A, relevant, document, must, minimum, identif..."
1,302,Poliomyelitis and Post-Polio,Is the disease of Poliomyelitis (polio) under ...,Relevant documents should contain data or outb...,"[Poliomyelitis, Post-Polio]","[Is, disease, Poliomyelitis, (, polio, ), cont...","[Relevant, documents, contain, data, outbreaks..."
2,303,Hubble Telescope Achievements,Identify positive accomplishments of the Hubbl...,Documents are relevant that show the Hubble te...,"[Hubble, Telescope, Achievements]","[Identify, positive, accomplishments, Hubble, ...","[Documents, relevant, show, Hubble, telescope,..."
3,304,Endangered Species (Mammals),Compile a list of mammals that are considered ...,Any document identifying a mammal as endangere...,"[Endangered, Species, (, Mammals, )]","[Compile, list, mammals, considered, endangere...","[Any, document, identifying, mammal, endangere..."
4,305,Most Dangerous Vehicles,"Which are the most crashworthy, and least cras...",A relevant document will contain information o...,"[Most, Dangerous, Vehicles]","[Which, crashworthy, ,, least, crashworthy, ,,...","[A, relevant, document, contain, information, ..."
...,...,...,...,...,...,...,...
245,696,safety plastic surgery,Find documents that discuss the safety of or t...,Relevant document must refer to a safety issue...,"[safety, plastic, surgery]","[Find, documents, discuss, safety, hazards, co...","[Relevant, document, must, refer, safety, issu..."
246,697,air traffic controller,What are working conditions and pay for U.S. a...,Relevant documents tell something about workin...,"[air, traffic, controller]","[What, working, conditions, pay, U.S., air, tr...","[Relevant, documents, tell, something, working..."
247,698,literacy rates Africa,What are literacy rates in African countries?,A relevant document will contain information a...,"[literacy, rates, Africa]","[What, literacy, rates, African, countries, ?]","[A, relevant, document, contain, information, ..."
248,699,term limits,What are the pros and cons of term limits?,Relevant documents reflect an opinion on the v...,"[term, limits]","[What, pros, cons, term, limits, ?]","[Relevant, documents, reflect, opinion, value,..."


In [5]:
bm25 = BM25Okapi(trec['Description_tokenized'], k1=1.2, b=0.75) # replace with corpuls['text'] in final 

In [6]:
bm25_res = [[x.Number, bm25.get_scores(x.Title_tokenized)] for x in trec.itertuples()] #replace Number with corpuls['Docid'] in final

In [7]:
bm25_idx_value = [[res[0], [[i, x] for i, x in enumerate(res[1]) if float(x) != 0]] for res in bm25_res]

In [8]:
arr = np.array(bm25_idx_value)

In [123]:
bm25_search_formatted = pd.Series(arr[:,1], index=arr[:,0]).apply(pd.Series).stack()
bm25_search_formatted = bm25_search_formatted.reset_index()
bm25_search_formatted.to_csv('./data/variable_storage/bm25_search_formatted.csv', header=['Topic', 'Docid', 'Value'], index=False)

In [128]:
bm25_search_formatted = pd.read_csv('./data/variable_storage/bm25_search_formatted.csv')

def makeArrayP1(text):
    text = text[1:-1]
    res = np.fromstring(text,sep=', ')
    return res[0]

def makeArrayP2(text):
    text = text[1:-1]
    res = np.fromstring(text,sep=', ')
    return res[1]

files = bm25_search_formatted.Value.apply(makeArrayP1)
vals = bm25_search_formatted.Value.apply(makeArrayP2)
bm25_search_formatted['Docid'] = files
bm25_search_formatted['Value'] = vals

bm25_search_formatted.Docid = trec.loc[bm25_search_formatted.Docid].Number.values

### Normalize Values

In [129]:
min_val = min(bm25_search_formatted.Value)
max_val =  max(bm25_search_formatted.Value)

bm25_search_formatted.Value = (bm25_search_formatted.Value - min_val) / (max_val - min_val)